# Карташов Никита Андреевич, гр. 0373

## Теория принятия решений. Вариант 140


### Задача 2

Технология краудсорсинга позволяет привлекать широкий круг людей к задачам сбора и обработки информации. Заказчики могут размещать на платформе краудсорсинга задания, назначая за них определенное вознаграждение, а пользователи платформы (исполнители) выполняют эти задания и получают назначенное вознаграждение. Примерами таких платформ являются Amazon Mechanical Turk, Яндекс.Толока. Одной из разновидностей краудсорсинга является пространственный краудсорсинг, при котором задания имеют пространственную привязку (например, с помощью системы пространственного краудсорсинга можно сделать фотографию определенной географической локации в определенный момент).

В данной задаче рассматривается один из центральных компонентов платформы пространственного краудсорсинга, осуществляющий распределение заданий среди исполнителей.

Задано определенное множество заданий (табл. 1). Каждое задание имеет пространственную привязку. Одной из функций подобной привязки являтся идентификация исполнителей, находящихся рядом с заданием. Для простоты будем полагать, что задания и исполнители находятся в городе Гипподамовой системы, что позволяет использовать локальную систему координат и оценивать расстояние с помощью манхеттенской метрики. Задания считаются достаточно простыми, чтобы любое из них могло быть выполнено любым участником.

В заданный момент в системе активно определенное множество участников (табл. 2). Для каждого участника задано текущее положение в локальной системе координат, а также текущий рейтинг (в диапазоне от 0 до 1 - чем больше, тем более надежным является исполнитель).

Вознаграждение исполнителя определяется его рейтингом. В частности, задано два граничных уровня рейтинга $Th_{M}=0.3$ (средний) и $Th_{H}=0.7$ (высокий). Участники с высоким рейтингом ($r_{i} \geq Th_{H}$) получают вознаграждение 110 денежных единиц (ДЕ) за каждое задание, участники со средним рейтингом ($Th_{M} \leq r_{i} \lt Th_{H}$) получают вознаграждение 60 ДЕ.
Участники с рейтингом ниже $Th_{M}$ считаются заблокированными и к выполнению заданий не привлекаются.

Бюджет заказчика на выполнение заданного набора заданий составляет 320 ДЕ.

Требуется:

1. Считая качество выполнения задания численно равным рейтингу исполнителя, найти распределение (всех) заданий по исполнителям, максимизирующее суммарное качество выполнения заданий.

2. Модифицировать целевую функцию с учетом расстояния между заданием и исполнителем (считая, что ожидаемое качество выполнения задания убывает как логарифм от расстояния) и найти распределение заданий с учетом расстояния.


<div style="text-align: center;"><i>Таблица 1: Задания</i></div>

| Идентификатор | x  | y  |
|---------------|----|----|
| 1             | 9  | 11 |
| 2             | 10 | 4  |
| 3             | 9  | 8  |
| 4             | 1  | 12 |


<div style="text-align: center;"><i>Таблица 2: Исполнители</i></div>

| Идентификатор | x  | y  | Рейтинг |
|---------------|----|----|---------|
| 1             | 17 | 16 | 0.93    |
| 2             | 10 | 10 | 0.35    |
| 3             | 7  | 2  | 0.78    |
| 4             | 15 | 18 | 0.75    |
| 5             | 6  | 14 | 0.83    |
| 6             | 11 | 7  | 0.23    |
| 7             | 14 | 15 | 0.73    |
| 8             | 18 | 0  | 0.32    |
| 9             | 17 | 9  | 0.91    |
| 10            | 15 | 14 | 0.26    |


### Выполним постановку задачи:
Пусть $N$ - количество заданий,<br>
$M$ - количество участников,<br>
$B$ - имеющийся бюджет,<br>
$x_{ij}$ - <i>бинарная</i> переменная, принимающая значение $1$ в случае назначения задания $i$ участнику $j$ и $0$ в противном случае,<br>
$c_{ij}$ - стоимость выполнения задания $i$ участником $j$, причём участники с рейтингом $\geq 0.7$ получают вознаграждение 110 ДЕ, участники с рейтингом в диапазоне $[0.3, 0.7)$ получают 60 ДЕ, а участники с рейтингом ниже $0.3$ не привлекаются к выполнению заданий;<br>
$r_{j}$ - рейтинг участника $j$,<br>
$k_{ij}$ - дополнительный коэффициент перед $r_{j}$, принимающий значение $1$ в случае, если соответствующий рейтинг $r_{j} \geq 0.3$ и $0$ в противном случае;<br>
$k_{ij}$, умноженные на соответствующие им $r_{j}$, образуют ожидаемое качество выполнения задания $i$ участником $j$.<br>

Целевая функция:
$\displaystyle\sum_{i=1}^N\displaystyle\sum_{j=1}^M{k_{ij}*r_{j}*x_{ij}} \to max$<br>
Ограничения:<br>
$\displaystyle\sum_{i=1}^N\displaystyle\sum_{j=1}^M{c_{ij}*x_{ij}} \leq B$<br>
$\displaystyle\sum_{i=1}^N{x_{ij}} \leq 1$ для $j \isin \{1, 2, ..., M\}$<br>
$\displaystyle\sum_{i=1}^N\displaystyle\sum_{j=1}^M{x_{ij}} = N$<br>
$\displaystyle\sum_{i=1}^N\displaystyle\sum_{j=1}^M{(1-k_{ij})*r_{j}*x_{ij}} = 0$<br>
$\displaystyle\sum_{j=1}^M{x_{ij}} = 1$ для $i \isin \{1, 2, ..., N\}$<br>

### Пункт 1.

#### 1.1 Импортируем необходимые для решения модули:


In [42]:
import math
import numpy as np
from cvxopt import matrix, glpk

#### 1.2 Введём начальные данные:


In [43]:
tasks = [[9, 11], [10, 4], [9, 8], [1, 12]]
budget = 340.0
users = [
    {"coords": [17, 16], "rating": 0.93},
    {"coords": [10, 10], "rating": 0.35},
    {"coords": [7, 2], "rating": 0.78},
    {"coords": [15, 18], "rating": 0.75},
    {"coords": [6, 14], "rating": 0.83},
    {"coords": [11, 7], "rating": 0.23},
    {"coords": [14, 15], "rating": 0.73},
    {"coords": [18, 0], "rating": 0.32},
    {"coords": [17, 9], "rating": 0.91},
    {"coords": [15, 14], "rating": 0.26},
]

#### 1.3 Рассчитаем, какие пользователи могут быть допущены к выполнению задания, а какие нет. Также рассчитаем стоимость их услуг:


In [44]:
activeUsers = matrix([users[i]["rating"] for i in range(len(users))], tc="d").T
availableUsers = matrix(
    [[activeUsers[i] if activeUsers[i] >= 0.3 else 0]
        for i in range(len(activeUsers))],
    tc="d",
)
unavailableUsers = activeUsers - availableUsers
cost = matrix(
    [
        (110 if activeUsers[i] >= 0.7 else 60) if (
            activeUsers[i] >= 0.3) else (0)
        for i in range(len(activeUsers))
    ],
    tc="d",
)

#### 1.4 Составим матрицы для ограничений - одно задание может выполнять только один участник, а также один участник не может выполнять больше одного задания:

In [45]:
volume = len(tasks) * len(users)
oneManToTask = np.array([])
for i in range(0, volume, len(users)):
    temp = np.concatenate(
        (np.zeros(i), np.ones(len(users)), np.zeros(volume - len(users) - i))
    )
    if i == 0:
        oneManToTask = np.copy(temp)
    else:
        oneManToTask = np.vstack((oneManToTask, temp))

In [46]:
oneTaskMax = np.array([])
for i in range(len(users)):
    temp = np.concatenate(
        (np.zeros(i), np.ones(1), np.zeros(len(users) - i - 1)))
    temp = np.tile(temp, len(tasks))
    if i == 0:
        oneTaskMax = np.copy(temp)
    else:
        oneTaskMax = np.vstack((oneTaskMax, temp))

#### 1.5 Составим целевую функцию и набор ограничений:


In [47]:
targetFunction = matrix([availableUsers.T for i in range(len(tasks))], tc="d").T
inequalityLeft = matrix([[cost for i in range(len(tasks))], [matrix(oneTaskMax).T]], tc="d").T
inequalityRight = matrix([budget, matrix(np.ones(len(users)))], tc="d")
equalityLeft = matrix(
    [
        [matrix([[1] for i in range(len(tasks) * len(activeUsers))]).T],
        [matrix([unavailableUsers.T for i in range(len(tasks))])],
        [matrix(oneManToTask).T],
    ],
    tc="d",
).T
equalityRight = matrix([len(tasks), 0, matrix(np.ones(len(tasks)))], tc="d")

#### 1.6 С помощью солвера "glpk" решим задачу по максимизации целевой функции с учётом всех ограничений. Принимаем во внимание, что получаемые нами переменные $x_{i}$ должны представлять собой булевый тип:


In [48]:
(status, x) = glpk.ilp(
    c=-targetFunction,
    G=inequalityLeft,
    h=inequalityRight,
    A=equalityLeft,
    b=equalityRight,
    B=set(range(volume)),
)

#### 1.7 Создадим функцию для вывода:

In [49]:
def printResult(resultStatus, resultArray):
    print(f"Статус: {resultStatus}")
    if resultStatus == 'optimal':
        print(resultArray)
        print("Распределение по заданиям:")
        for i in range(len(tasks)):
            resultSlice = resultArray[i*len(users) : i*len(users) + len(users)]
            for j in range(len(users)):
                if(resultSlice.T[j] > 0):
                    print(f"\nЗадание {1+i}: Участник {1+j}")
                    print(f"Рейтинг участника: {users[j]['rating']}")
                    print(f"Эффективность: {round(targetFunction[len(users)*i+j], 4)}")
                    print(f"Оплата: {cost[j]}")
                    break
    else:
        print("Оптимальное решение не было найдено.")

#### 1.8 Выведем и проинтерпретируем результат:


In [50]:
printResult(status, x)

Статус: optimal
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]

Распределение по заданиям:

Задание 1: Участник 2
Рейтинг участника: 0.35
Эффективность: 0.35
Оплата: 60.0

Задание 2: Участник 8
Рейтинг участника: 0.32
Эффективность: 0.32
Оплата: 60.0

Задание 3: Участник 1
Рейтинг участника: 0.93
Эффективность: 0.93
Оплата: 110.0

Задание 4: Участник 9
Рейтинг участника: 0.91
Эффективность: 0.91
Оплата: 110.0


### Пункт 2.

#### 2.1 Создадим функцию подсчёта коэффициента, зависящего от расстояния между заданием и участником:

In [51]:
def distCoefficient(location1, location2):
    logBase = 2
    distance = 0.0
    for i in range(len(location1)):
        distance += abs(location1[i] - location2[i])
    return (1 if (distance < logBase) else (1/math.log(distance, logBase))**1)

#### 2.2 Рассчитаем коэффициенты эффективности выполнения заданий с учётом расстояния до них и модифицируем функцию цели с учётом этих коэффициентов:

In [52]:
coefficients = np.array([])
for i in range(len(tasks)):
    for j in range(len(users)):
        coefficients = np.append(coefficients, distCoefficient(users[j]['coords'], tasks[i]))
targetFunction = matrix([targetFunction[i]*coefficients[i] for i in range(volume)], tc='d')

#### 2.3 С помощью солвера "glpk" решим задачу по максимизации модифицированной целевой функции:

In [53]:
(status2, x2) = glpk.ilp(
    c=-targetFunction,
    G=inequalityLeft,
    h=inequalityRight,
    A=equalityLeft,
    b=equalityRight,
    B=set(range(volume)),
)

#### 2.4 Выведем и проинтерпретируем результат:

In [54]:
printResult(status2, x2)

Статус: optimal
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]

Распределение по заданиям:

Задание 1: Участник 2
Рейтинг участника: 0.35
Эффективность: 0.35
Оплата: 60.0

Задание 2: Участник 3
Рейтинг участника: 0.78
Эффективность: 0.3359
Оплата: 110.0

Задание 3: Участник 8
Рейтинг участника: 0.32
Эффективность: 0.0783
Оплата: 60.0

Задание 4: Участник 5
Рейтинг участника: 0.83
Эффективность: 0.2957
Оплата: 110.0
